<a href="https://colab.research.google.com/github/srujan-b/World-best-XI-from-ICC-T20-world-cup-2022-australia/blob/main/webscrappingdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
from bs4 import BeautifulSoup
import requests
import smtplib
import time
from datetime import datetime
import re

In [3]:
# Url defination
url = 'https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/match-schedule-fixtures-and-results'

# Header for web scrapping
headers = {"User-Agen":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Brave/96.0.4664.110","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br, zstd", "Accept-Language": "en-US,en","DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}



page = requests.get(url,headers = headers)

In [8]:
resultPage = BeautifulSoup(page.content, "html.parser")

[<div class="ds-text-compact-xs ds-font-bold ds-w-24">Sun, 16 Oct '22</div>, <div class="ds-text-compact-xs ds-font-bold ds-w-24"></div>, <div class="ds-text-compact-xs ds-font-bold ds-w-24">Mon, 17 Oct '22</div>, <div class="ds-text-compact-xs ds-font-bold ds-w-24"></div>, <div class="ds-text-compact-xs ds-font-bold ds-w-24">Tue, 18 Oct '22</div>, <div class="ds-text-compact-xs ds-font-bold ds-w-24"></div>, <div class="ds-text-compact-xs ds-font-bold ds-w-24">Wed, 19 Oct '22</div>, <div class="ds-text-compact-xs ds-font-bold ds-w-24"></div>, <div class="ds-text-compact-xs ds-font-bold ds-w-24">Thu, 20 Oct '22</div>, <div class="ds-text-compact-xs ds-font-bold ds-w-24"></div>, <div class="ds-text-compact-xs ds-font-bold ds-w-24">Fri, 21 Oct '22</div>, <div class="ds-text-compact-xs ds-font-bold ds-w-24"></div>, <div class="ds-text-compact-xs ds-font-bold ds-w-24">Sat, 22 Oct '22</div>, <div class="ds-text-compact-xs ds-font-bold ds-w-24"></div>, <div class="ds-text-compact-xs ds-font-b

In [151]:
# Extracting match Dates

dates = resultPage.find_all("div", {"class": "ds-text-compact-xs ds-font-bold ds-w-24"})

date = []
for i in dates:

  if i.get_text():
    date_string = i.get_text(strip=True).split(',')[1]
    date_object = datetime.strptime(date_string, " %d %b '%y")
    formatted_date = date_object.strftime("%Y-%m-%d")
    date.append(formatted_date)
  else:
    date.append(formatted_date)

In [152]:
# Extracting match numbers

matchNumbers = resultPage.find_all("span", {"class": "ds-text-tight-s ds-font-medium ds-text-typo"})

matchNum = []
for i in matchNumbers:
  matchDetails = i.get_text(strip =True).split(",")[0].split()
  if matchDetails[1] == 'Match':
    pattern = r'\d+'
    numbers = int(re.search(pattern, matchDetails[0]).group())
    matchNum.append(numbers)
  else:
    matchNum.append(" ".join(matchDetails[0:-1]))

In [153]:
# Extracting City

place = resultPage.find_all("div", {"class": "ds-text-tight-s ds-font-regular ds-truncate ds-text-typo-mid3"})

city = []

for i in place:
  if i.contents[3]:
    city.append(i.contents[3])


In [157]:
# extracting teams
team1Find = resultPage.find_all("p", {"class": "ds-text-tight-m ds-font-bold ds-capitalize ds-truncate"})

teamName = {'England':'ENG', 'India':'IND', 'Scotland':'SCOT', 'Ireland':'IRE', 'Sri Lanka':'SL', 'Zimbabwe':'ZIM', 'Afghanistan':'AFG', 'Australia':'AUS', 'Pakistan':'PAK', 'United Arab Emirates':'UAE', 'Bangladesh':'BAN', 'South Africa':'SA', 'Netherlands':'NED', 'New Zealand':'NZ', 'Namibia':'NAM', 'West Indies':'WI'}
team1 = []
team2 = []
team1SF = []
team2SF = []
a = 1
for i in team1Find:
  if a % 2 != 1:
    team = i.get_text(strip=True)
    team1.append(team)
    team1SF.append(teamName[team])
    a +=1
  else:
    team = i.get_text(strip=True)
    team2.append(team)
    team2SF.append(teamName[team])
    a+=1


In [159]:
# scores of the match

scores_class_divs = resultPage.find_all("div", {"class": "ds-flex ds-flex-col ds-mt-2 ds-mb-2"})

team1Score = []
team2Score = []

a = 1
for div in scores_class_divs:

  if div.find_all("strong", class_=""):
    bothTeamScore = div.find_all("strong", class_="")
    team1s = bothTeamScore[0].get_text(strip=True)
    team2s = bothTeamScore[1].get_text(strip=True)
    team1Score.append(team1s)
    team2Score.append(team2s)
  else:
    team1Score.append(" ")
    team2Score.append(" ")




In [168]:
# Results

resultsClassDivs = resultPage.find_all("p", {"class": "ds-text-tight-s ds-font-medium ds-line-clamp-2 ds-text-typo"})

results = []


for i in resultsClassDivs:
  results.append(i.find("span").get_text(strip=True))



In [173]:
#scorecardURl

scoreCardUrlDivs = resultPage.find_all("a", {"class": "ds-no-tap-higlight"})

url = []
for i in scoreCardUrlDivs:

  url.append("https://www.espncricinfo.com"+i.get("href"))


In [176]:
import pandas as pd
finalList = zip(date,matchNum,city,team1,team1SF,team1Score,team2,team2SF,team2Score,results,url)

df = pd.DataFrame(finalList, columns=('MatchDate','MatchNumber','City','Team1','Team1SF','Team1Score','Team2','Team2SF','Team2Score','Results','url'))


In [177]:
df

,MatchDate,MatchNumber,City,Team1,Team1SF,Team1Score,Team2,Team2SF,Team2Score,Results,url
0,2022-10-16,1,Geelong,Sri Lanka,SL,163/7,Namibia,NAM,108,Namibia won by 55 runs,https://www.espncricinfo.com/series/indian-pre...
1,2022-10-16,2,Geelong,Netherlands,NED,111/8,United Arab Emirates,UAE,112/7,Netherlands won by 3 wickets (with 1 ball rema...,https://www.espncricinfo.com/series/usa-vs-can...
2,2022-10-17,3,Hobart,West Indies,WI,160/5,Scotland,SCOT,118,Scotland won by 42 runs,https://www.espncricinfo.com/series/indian-pre...
3,2022-10-17,4,Hobart,Ireland,IRE,174/7,Zimbabwe,ZIM,143/9,Zimbabwe won by 31 runs,https://www.espncricinfo.com/series/icc-men-s-...
4,2022-10-18,5,Geelong,Netherlands,NED,121/6,Namibia,NAM,122/5,Netherlands won by 5 wickets (with 3 balls rem...,https://www.espncricinfo.com/series/icc-men-s-...
5,2022-10-18,6,Geelong,United Arab Emirates,UAE,152/8,Sri Lanka,SL,73,Sri Lanka won by 79 runs,https://www.espncricinfo.com/series/icc-men-s-...
6,2022-10-19,7,Hobart,Ireland,IRE,176/5,Scotland,SCOT,180/4,Ireland won by 6 wickets (with 6 balls remaining),https://www.espncricinfo.com/series/icc-men-s-...
7,2022-10-19,8,Hobart,Zimbabwe,ZIM,153/7,West Indies,WI,122,West Indies won by 31 runs,https://www.espncricinfo.com/series/icc-men-s-...
8,2022-10-20,9,Geelong,Netherlands,NED,162/6,Sri Lanka,SL,146/9,Sri Lanka won by 16 runs,https://www.espncricinfo.com/series/icc-men-s-...
9,2022-10-20,10,Geelong,Namibia,NAM,148/3,United Arab Emirates,UAE,141/8,U.A.E. won by 7 runs,https://www.espncricinfo.com/series/icc-men-s-...


In [174]:
for i in zip(date,matchNum,city,team1,team1SF,team1Score,team2,team2SF,team2Score,results,url):

  print(i)

('2022-10-16', 1, 'Geelong', 'Sri Lanka', 'SL', '163/7', 'Namibia', 'NAM', '108', 'Namibia won by 55 runs', 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/rajasthan-royals-vs-gujarat-titans-24th-match-1426262/live-cricket-score')
('2022-10-16', 2, 'Geelong', 'Netherlands', 'NED', '111/8', 'United Arab Emirates', 'UAE', '112/7', 'Netherlands won by 3 wickets (with 1 ball remaining)', 'https://www.espncricinfo.com/series/usa-vs-canada-2024-1425115/united-states-of-america-vs-canada-3rd-t20i-1425123/live-cricket-score')
('2022-10-17', 3, 'Hobart', 'West Indies', 'WI', '160/5', 'Scotland', 'SCOT', '118', 'Scotland won by 42 runs', 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/mumbai-indians-vs-royal-challengers-bengaluru-25th-match-1426263/live-cricket-score')
('2022-10-17', 4, 'Hobart', 'Ireland', 'IRE', '174/7', 'Zimbabwe', 'ZIM', '143/9', 'Zimbabwe won by 31 runs', 'https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-129